# Multi Task Model

## Imports

In [1]:
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNet

from loadData import createImageDataset

## Model parameters

In [2]:
# Log parameters
model_name = 'multiTaskModel'
savedModelPath = f'../../log/saved_models/{model_name}'
tb_log_dir = f'../../log/tensorboard/{model_name}'
cp_filepath = f'../../log/cps/{model_name}/latest_weights.h5'

# Dynamic hyperparameters
learningRate = 0.001
doDataAugmentation = True
dropoutRate = 0.25
width_multiplier = 1
depth_multiplier = 1

# Training parameters
batch_size = 32
epochs = 10

## Model callbacks

In [3]:
callbacks = [
    # Checkpoint callback                    
    keras.callbacks.ModelCheckpoint(
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

    # Tensorboard callback
    keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=2,
                    verbose=1)
]

## Data parameters

In [4]:
image_height = 224
image_width = 224

## Data Augmentation

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomBrightness(0.2),
    ]
)

## Creating the training datasets

In [6]:
training_data, training_labels = createImageDataset('../../data/m3/training', (image_height, image_width))
training_face_labels = training_labels[:, 0]
training_mask_labels = training_labels[:, 1]
training_age_labels = training_labels[:, 2]

Found 14 files belonging to 1 classes.


## Creating the test dataset

In [7]:
test_data, test_labels = createImageDataset('../../data/m3/test', (image_height, image_width))
test_face_labels = test_labels[:, 0]
test_mask_labels = test_labels[:, 1]
test_age_labels = test_labels[:, 2]

Found 14 files belonging to 1 classes.


d:\Master Studium Informatik\3. Semester\Advanced Deep Learning\Projekt\dl_faceRecognition\src\m3\loadData.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(images), np.array(labels)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Load the ResNet model

In [8]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
mobilenet = MobileNet(
                input_shape=(image_height, image_width, 3), # Optional shape tuple, only to be specified if include_top is False
                alpha=width_multiplier, # Controls the width of the network. (Width multiplier)
                depth_multiplier=depth_multiplier, # Depth multiplier for depthwise convolution. (Resolution multiplier)
                dropout=dropoutRate, # Dropout rate. Default to 0.001.
                weights="imagenet",
                input_tensor=None,
                pooling='avg', # Optional pooling mode for feature extraction when include_top is False. (None, avg, max)
                include_top=False
                )
           
# Freeze the base model
mobilenet.trainable = False

inputs = keras.Input(shape=(image_height, image_width, 3))

# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
base_model = mobilenet(inputs, training=False)

## Creating Task 1 (Face Detection) Top Model

In [9]:
# Add Dense layer
face_head = tf.keras.layers.Dense(512, activation='relu')(base_model)

# Final layer for binary classification
face_outputs = keras.layers.Dense(1, activation='sigmoid', name='face_output')(face_head)

## Creating Task 2 (Mask Detection) Top Model

In [10]:
# Add Dense layer
mask_head = tf.keras.layers.Dense(512, activation='relu')(base_model)

# Final layer for binary classification
mask_outputs = keras.layers.Dense(1, activation='sigmoid', name='mask_output')(mask_head)

## Creating Task 3 (Age Prediction) Top Model

In [11]:
# Add Dense layer
age_head = tf.keras.layers.Dense(1024, activation='relu')(base_model)

# Final layer for binary classification
age_outputs = keras.layers.Dense(120, activation='softmax', name='age_output')(age_head)

## Creating and compiling the final model

In [12]:

model = keras.Model(inputs, [face_outputs, mask_outputs, age_outputs])

# Using a joint loss function for the three tasks:
# [ Loss = gamma * Loss_task1 + gamma * Loss_task2 + gamma * Loss_task3 ]
# Because every task is dependant on every other task, the model receives the loss of every task when gamma > 0

gamma = 0.33

model.compile(
            optimizer=keras.optimizers.Adam(), # Learning Rate?
            loss={'face_output': keras.losses.BinaryCrossentropy(), 
                  'mask_output': keras.losses.BinaryCrossentropy(),
                  'age_output': keras.losses.SparseCategoricalCrossentropy()},
            loss_weights={'face_output': gamma, 
                        'mask_output': gamma,
                        'age_output': gamma}, 
            metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mobilenet_1.00_224 (Functional  (None, 1024)        3228864     ['input_3[0][0]']                
 )                                                                                                
                                                                                                  
 dense (Dense)                  (None, 512)          524800      ['mobilenet_1.00_224[1][0]']     
                                                                                              

## Training the model with the dataset

In [14]:
history = model.fit(
            x=training_data,
            y={'face_output': training_face_labels, 
             'mask_output': training_mask_labels,
             'age_output': training_age_labels},
            batch_size=batch_size,
            epochs=epochs, 
            callbacks=callbacks,
            validation_split=0.2,
            shuffle=True
        )

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.0029 - face_output_loss: 2.5924e-10 - mask_output_loss: 6.1243e-10 - age_output_loss: 0.0088 - face_output_accuracy: 1.0000 - mask_output_accuracy: 1.0000 - age_output_accuracy: 1.0000
Epoch 1: saving model to ../../log/cps/multiTaskModel\latest_weights.h5
1/1 [==============================] - 5s 5s/step - loss: 0.0029 - face_output_loss: 2.5924e-10 - mask_output_loss: 6.1243e-10 - age_output_loss: 0.0088 - face_output_accuracy: 1.0000 - mask_output_accuracy: 1.0000 - age_output_accuracy: 1.0000 - val_loss: 0.0039 - val_face_output_loss: 6.3391e-12 - val_mask_output_loss: 2.6650e-11 - val_age_output_loss: 0.0119 - val_face_output_accuracy: 1.0000 - val_mask_output_accuracy: 1.0000 - val_age_output_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.0017 - face_output_loss: 8.6023e-11 - mask_output_loss: 2.0018e-10 - age_output_loss: 0.0052 - face_output_accuracy: 1.0000 - mask_output_a

## Save the model

In [ ]:
model.save(savedModelPath)

## Test model with test dataset

In [ ]:
results = model.evaluate(
                x=test_data,
                y={'face_output': test_face_labels, 
                'mask_output': test_mask_labels,
                'age_output': test_age_labels},
)

print(f'Loss: {results[0]}; Accuracy: {results[1]}')

## Predict new images

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../../data/m3/test/FaceTest/000001.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)